In [2]:
import warnings
import pickle
import pandas as pd
import numpy as np
from sklearn.exceptions import UndefinedMetricWarning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split

warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

data = pd.read_hdf('..//data//model//data_model_clus.h5')

In [76]:
X = data[[
    
 'Tenderloin',
 'South of Market',
 'Downtown',
 'Inner Mission',
 'Van Ness/Civic Center',
 'Duboce Triangle',
 'Haight Ashbury',
 'Miraloma Park',
 'Financial District/Barbary Coast',
 'Silver Terrace', 
    
 'winter',
 'spring',
 'summer',
 'fall',
 'hour_0',
 #'hour_6',
 'hour_12',
 #'hour_18',
 'dow_0',
 'dow_1',
 'dow_2',
 'dow_3',
 'dow_4',
 'dow_5',
 'dow_6',
 'UNIVPROX',
 'SIGNALIZED',
 'PKGMETERS',
 'MAXPCTSLPE',
 'MODEL6_VOL',
 'HH_PEDMODE',
 'PCOL_04_09',
 'PCOL_RATE',
 'HH_income',
 'total_pop',
 'unemp_20_24',
 'unemp_25_44',
 'unemp_45_54',
 'unemp_55_64',
 'white',
 'black',
 'asian',
 'other'
]]

y = data['poop']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

X_train_imp = Imputer().fit_transform(X_train)
X_train_scale = StandardScaler().fit_transform(X_train_imp)

X_test_imp = Imputer().fit_transform(X_test)
X_test_scale = StandardScaler().fit_transform(X_test_imp)

In [77]:
small_rf = RandomForestClassifier(n_jobs=-1,
                                  class_weight={1: 5, 0: 1},
                                  criterion='gini', 
                                  max_depth=6, 
                                  n_estimators=115)
small_rf = small_rf.fit(X_train_scale, y_train)

In [ ]:
with open('..//models//random_forest.pkl', 'wb') as fid:
    pickle.dump(gnb, fid)